In [8]:
# Célula 0 — Instalar dependências
! pip install pandas matplotlib openpyxl os



Defaulting to user installation because normal site-packages is not writeable
  Using cached pandas-3.0.0-cp314-cp314-win_amd64.whl.metadata (19 kB)
  Using cached matplotlib-3.10.8-cp314-cp314-win_amd64.whl.metadata (52 kB)
  Using cached openpyxl-3.1.5-py2.py3-none-any.whl.metadata (2.5 kB)


ERROR: Could not find a version that satisfies the requirement os (from versions: none)

[notice] A new release of pip is available: 25.2 -> 26.0.1
[notice] To update, run: C:\Program Files\Python314\python.exe -m pip install --upgrade pip
ERROR: No matching distribution found for os


In [9]:
# Célula 1 — Imports
import os
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path


In [10]:
# Célula 2 — Conferir diretório atual
os.getcwd()
os.listdir()

['analisebase.ipynb',
 'br_seeg_emissoes_brasil.csv',
 'br_seeg_emissoes_brasil.xlsx']

In [ ]:
# Célula 3 — Montar caminho do arquivo com os
caminho = os.path.join("br_seeg_emissoes_brasil.xlsx")
df = pd.read_excel(caminho)


In [ ]:
# Célula 4 — Inspeção inicial
df.head()
df.columns
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 454850 entries, 0 to 454849
Data columns (total 12 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   ano                  454850 non-null  int64  
 1   nivel_1              454850 non-null  object 
 2   nivel_2              454850 non-null  object 
 3   nivel_3              454850 non-null  object 
 4   nivel_4              454850 non-null  object 
 5   nivel_5              454850 non-null  object 
 6   nivel_6              454850 non-null  object 
 7   tipo_emissao         454850 non-null  object 
 8   gas                  454850 non-null  object 
 9   atividade_economica  453100 non-null  object 
 10  produto              268100 non-null  object 
 11  emissao              406325 non-null  float64
dtypes: float64(1), int64(1), object(10)
memory usage: 41.6+ MB


In [ ]:
# Célula 5 — Ajustar tipos
df["ano"] = df["ano"].astype(int)
df["emissao"] = pd.to_numeric(df["emissao"], errors="coerce")

In [ ]:
# Célula 6 — Filtros 

# Exemplo: Apenas registros de emissão
# df = df[df["tipo_emissao"].str.contains("Emissão", case=False)]

# Apenas Agropecuária
# df = df[df["nivel_1"] == "Agropecuária"]



In [ ]:
# Célula 7 — Criar períodos de 5 em 5 anos
df["ano_5"] = (df["ano"] // 5) * 5
df["periodo"] = df["ano_5"].astype(str) + "-" + (df["ano_5"] + 4).astype(str)

df[["ano", "periodo"]].head(10)


In [ ]:
# Célula 9 — Pivot para gráfico
df_pivot = (
    df_group
    .pivot(index="periodo", columns="tipo_emissao", values="emissao")
    .fillna(0)
)


In [ ]:
# Célula 10 — Gráfico principal (coluna empilhada)
df_pivot.plot(kind="bar", stacked=True, figsize=(12, 6))

plt.xlabel("Período (5 anos)")
plt.ylabel("Emissão")
plt.title("Emissões agrupadas de 5 em 5 anos por tipo de emissão")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
# Célula 11 — Gráfico de linhas

df_pivot.plot(kind="line", figsize=(12, 6), marker="o")

plt.xlabel("Período (5 anos)")
plt.ylabel("Emissão")
plt.title("Evolução das emissões (5 em 5 anos)")
plt.xticks(rotation=45)
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
# Célula 12 — Reduzir colunas do gráfico
colunas = [c for c in df_pivot.columns if "CO2" in c or "CH4" in c]
df_pivot[colunas].plot(kind="bar", stacked=True, figsize=(12,6))

plt.xlabel("Período (5 anos)")
plt.ylabel("Emissão")
plt.title("Emissões de CO₂e e CH₄ (5 em 5 anos)")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
# Célula 13 — Salvar gráfico
plt.figure(figsize=(12,6))
df_pivot.plot(kind="bar", stacked=True)

plt.xlabel("Período (5 anos)")
plt.ylabel("Emissão")
plt.title("Emissões agrupadas de 5 em 5 anos")
plt.tight_layout()
plt.savefig("emissoes_5_em_5.png", dpi=300)
plt.show()


In [ ]:
# Célula 14 — Exportar tabela final
df_pivot.to_excel("tabela_emissoes_5_em_5.xlsx")
